# Análisis Exploratorio: Penetración de Créditos en Factura de Gas

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)


In [ ]:
# Cargar datasets
df_pot = pd.read_csv('../data/datosFNB.csv', sep=';', encoding='ISO-8859-1')
df_ven = pd.read_csv('../data/HistoricoVentas.csv', encoding='utf-8')
df_res = pd.read_csv('../data/DatosResueltos.csv', sep=';', encoding='cp1252')

In [ ]:
# Vista previa de los datos
df_pot.head()

In [ ]:
# Limpieza de potenciales
def limpiar_potencial(df):
    df['nombreDepartamento'] = df['nombreDepartamento'].str.upper()
    df['municipio'] = df['municipio'].str.upper()
    df['Cupo'] = pd.to_numeric(df['Cupo'], errors='coerce')
    df['estrato'] = pd.to_numeric(df['estrato'], errors='coerce')
    return df

pot_clean = limpiar_potencial(df_pot)

In [ ]:
# Limpieza de ventas
def limpiar_ventas(df):
    df['fechaVenta'] = pd.to_datetime(df['fechaVenta'], errors='coerce')
    df['nombreDepartamento'] = df['nombreDepartamento'].str.upper()
    df['municipio'] = df['municipio'].str.upper()
    df['valorTotal'] = pd.to_numeric(df['valorTotal'], errors='coerce')
    return df[df['estadoVenta'] == 'Entregado']

ven_clean = limpiar_ventas(df_ven)

In [ ]:
# Análisis por distribuidora
dist_pot = pot_clean.groupby('Distribuidora').agg(clientes_potenciales=('Codigo', 'count')).reset_index()
dist_ven = ven_clean.groupby('Canal').agg(clientes_con_credito=('documentoUsuario', 'nunique')).reset_index()
dist_ven.rename(columns={'Canal': 'Distribuidora'}, inplace=True)
penetracion = pd.merge(dist_pot, dist_ven, on='Distribuidora', how='left')
penetracion['tasa_penetracion'] = (penetracion['clientes_con_credito'] / penetracion['clientes_potenciales']) * 100
penetracion

In [ ]:
# Visualización de penetración por distribuidora
plt.figure(figsize=(10,6))
sns.barplot(data=penetracion, x='Distribuidora', y='tasa_penetracion')
plt.title('Tasa de Penetración por Distribuidora')
plt.show()

In [ ]:
# Análisis completado.